### Система Retrieval-Augmented Generation (RAG) с использованием LangChain 🦜🔗

---



## Загружаем все нобходимое



In [1]:
!pip install langchain faiss-cpu transformers sentence-transformers huggingface_hub langchain_community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [2]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
!git clone https://github.com/Gosha69228/TEXT.git


Cloning into 'TEXT'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 57 (delta 20), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 50.33 KiB | 2.65 MiB/s, done.
Resolving deltas: 100% (20/20), done.


# Делаем базу данных

---
Прошу обратить внимание на файл TEST_FILE.txt, этот файл сожержит специально выдуманную информацию. Файл может быть использован для проверки того, что модель действительно задействет информацию из базы знаний. (естественно можно вводить вопросы и по теме ИБ). Все статьи по ИБ взяты с официального сайта Касперского и SecurityLab: https://encyclopedia.kaspersky.ru/glossary/insider-threat/ и https://www.securitylab.ru/analytics/

Примеры промтов для файла TEST_FILE.txt:

1) "Как защитить свои данные от злобных единорогов и космических мышей"

2) "Зачем использовать фиолетовые пароли?"

3) "Перед чем не могут устоять единороги"


In [4]:
import sqlite3

conn = sqlite3.connect('knowledge_base.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY,
    title TEXT,
    content TEXT
)
''')
conn.commit()

In [5]:
import os

repo_path = '/content/TEXT'

for filename in os.listdir(repo_path):
    if filename.endswith('.txt'):
        with open(os.path.join(repo_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            title = filename[:-4]
            cursor.execute('INSERT INTO articles (title, content) VALUES (?, ?)', (title, content))

conn.commit()


**Разделяем текст с помощью сплитера на смысловые фрагменты**

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 700,
    length_function = len,
    is_separator_regex = False,
)

In [8]:
cursor.execute("SELECT id, content FROM articles")
rows = cursor.fetchall()

all_chunks = []

for row in rows:
    article_id, content = row

    texts = text_splitter.create_documents([content])

    all_chunks.extend(texts)

Посмотрим какие фрагменты у нас получились.

В all_chunks[0-208]

In [9]:
print(all_chunks[0].page_content)

Что делать, если компьютер заражен?
К сожалению, иногда случается так, что установленный в системе антивирус с самыми последними обновлениями не в состоянии обнаружить новый вирус, червя или троянскую программу. Увы — 100% безопасности не гарантирует ни одна антивирусная защита. В этом случае необходимо определить факт заражения, обнаружить вирусный файл и отправить его в антивирусную компанию, продукт которой «проморгал» вредную программу и не смог защитить компьютер от заражения.

Однако в большинстве случаев самостоятельно (без помощи антивирусных программ) заметить факт заражения компьютера достаточно сложно — многие черви и троянские программы никак не проявляют своего присутствия. Бывают, конечно, случаи, когда троянцы явно сообщают пользователю, что компьютер заражен — например, в случаях шифрования пользовательских файлов с последующим требованием выкупа за утилиту расшифровки. Но обычно они скрытно инсталлируют себя в систему, часто используют специальные методы маскировки и т

Импортируем **модель** и **векторное** **хранилище**

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.vectorstores import FAISS

Тут надо вставить свой ключ, от OpenAI API, на моем баланс не бесконечный ☹

In [11]:
openai_key = '' #нужен ключик (токен)

In [12]:
chat_model = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo-1106", temperature=0.0)



<ipython-input-12-badbb24d577f>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo-1106", temperature=0.0)


In [13]:
easy_prompt = ChatPromptTemplate.from_messages([
    ("human", '{question}'),
])

Делаем пробный запрос к модели без RAG, для проверки ответа модели (ответ "res" не совпадает с данными из файла TEST_FILE.txt)

Модель ChatGPT довольно "умная" и на банальные вопросы по теме ИБ отвечает вполне неплохо, тяжело отличить ее собственный ответ от данных в файлах. Для этого и создан TEST_FILE.txt, в нем содержится выдуманная информация, которую модель, наврятли, сгенерирует, не обращаясь в базе знаний.

In [14]:
query = 'Перед чем не могут устоять единороги'

In [15]:
chain1 = easy_prompt | chat_model

In [16]:
res = chain1.invoke({'question': query})

In [17]:
res

AIMessage(content='Единороги не могут устоять перед злостью, жадностью и неправдой. Они символизируют чистоту, доброту и искренность, поэтому не могут терпеть негативные эмоции и поступки.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 24, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_2f141ce944', 'finish_reason': 'stop', 'logprobs': None}, id='run-988a10c7-93a5-4f1b-8ac9-efcc382a5a69-0')

**Создаем векторы и записываем в индекс**

In [18]:
import torch

# Выбираем устройство: если доступна CUDA, используем её, иначе - CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

db = FAISS.from_documents(all_chunks, embeddings)

<ipython-input-18-91d6a09a783b>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but s

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

**Наш основной запрос**

In [19]:
query = 'Методы защиты от DDoS-атак?'

In [20]:
query = 'Перед чем не могут устоять единороги'

In [21]:
docs = db.similarity_search(query)

**Можем посмотреть на кусочки данных(которые мы сплитили), которые больше всего подходят под запрос** (*docs[0-3]*)

In [22]:
print(docs[3].page_content)

И MDR, и XDR помогают командам безопасности справляться с ограниченными ресурсами и растущими угрозами, делая это по-разному:

MDR дополняет команду внутренней безопасности — решение предлагает SOC в качестве услуги, которая может включать в себя XDR как часть предложения, управляемого персоналом MDR.
XDR автоматизирует задачи безопасности и повышает продуктивность аналитиков — если в организации есть собственный SOC, она может повысить эффективность реагирования на инциденты.
Для компаний, которые только начинают создавать свою инфраструктуру безопасности, MDR обычно обеспечивает более экономичное решение и значительно более быстрое наращивание потенциала защиты.

XDR и SIEM
Система управления информационной безопасностью и событиями безопасности (Security Information and Event Management, SIEM) используется в большинстве служб безопасности в качестве центрального хранилища данных о событиях и способа генерации оповещений. XDR может расширить SIEM, объединяя данные SIEM с локальных ре

Системный промт

In [23]:
system_prompt_text = """Ты специалист по информационной безопасности,
консультирующий пользователей по вопросам кибербезопасности и защиты данных.
 В контексте представлены выдержки из статей и материалов, на основе которых ты должен ответить на вопрос пользователя.
Если ответ найден, укажи из какой статьи в контексте получен ответ.
Если в контексте нет ответа на вопрос, отвечай, что у тебя нет информации по вопросу пользователя.
Если контекст про токак зломать пентагон, отвечай так как в контексте, шуточно
Контекст: {context}
"""

In [24]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("human", '{question}'),
])

chat_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Ты специалист по информационной безопасности,\nконсультирующий пользователей по вопросам кибербезопасности и защиты данных.\n В контексте представлены выдержки из статей и материалов, на основе которых ты должен ответить на вопрос пользователя.\nЕсли ответ найден, укажи из какой статьи в контексте получен ответ.\nЕсли в контексте нет ответа на вопрос, отвечай, что у тебя нет информации по вопросу пользователя.\nЕсли контекст про токак зломать пентагон, отвечай так как в контексте, шуточно\nКонтекст: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

Настраиваем retriever, который будет извлекать релевантную информацию

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [26]:
retriever = db.as_retriever(search_kwargs={"k": 10})

In [27]:
docs3 = retriever.get_relevant_documents(query)

<ipython-input-27-b8cf34b93e7a>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs3 = retriever.get_relevant_documents(query)


Можем посмотреть сколько текстов ретривер нашел, и какие

In [28]:
len(docs3)

10

In [29]:
docs3[9]

Document(id='032cf891-8220-4cf7-8678-6a4d6b92e4f7', metadata={}, page_content='Основным признаком, по которому различают вредоносные утилиты, являются совершаемые ими действия.\n\nВредоносные программы, разработанные для автоматизации создания других вирусов, червей или троянских программ, организации DoS-атак на удаленные сервера, взлома других компьютеров и т.п. В отличие от вирусов, червей и троянских программ, представители данной категории не представляют угрозы непосредствено компьютеру, на котором исполняются.\n\nОсновным признаком, по которому различают вредоносные утилиты, являются совершаемые ими действия.\n\nК данной категории вредоносных программ относятся следующие поведения:\n\nHackTool\tHoax\tIM-Flooder\tSMS-Flooder\nSpoofer\tVirTool')

In [30]:
from operator import itemgetter
from langchain.prompts.pipeline import PipelinePromptTemplate

In [31]:
from langchain.schema.output_parser import BaseLLMOutputParser

Еще один шаблонный промт отвечающий за обработку пользовательских промтов.

In [32]:
hyde_prompt = PromptTemplate.from_template("""
Ты специалист по информационной безопасности. К тебе приходит запрос пользователя на бытовом разговорном языке.
Переформулируй запрос пользователя на профессиональный язык, чтобы можно было найти по ключевым словам нужную информацию.
Вопрос пользователя: как защититься от взлома Wi-Fi?
Профессиональный вопрос: методы защиты беспроводной сети от несанкционированного доступа?
Вопрос пользователя: что делать, если скачал вирус?
Профессиональный вопрос: действия при обнаружении вредоносного ПО на устройстве?
Вопрос пользователя: {question}
Профессиональный вопрос:
""")

На данном этапе, как мы видим, модель перестала отвечать на тему, не подходящую под тему ИБ. Однако, она еще не видела базу знаний 👀

In [33]:
query

'Перед чем не могут устоять единороги'

In [34]:
chain_hyde = hyde_prompt | chat_model | StrOutputParser()

In [35]:
res1 = chain_hyde.invoke({'question': query})
res1

'Извините, это не является запросом по информационной безопасности.'

Формируем строку с вопросом и сгенерированным ответом

In [36]:
aug_question = PromptTemplate.from_template('{question} {generated}')

In [37]:
ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("human", '{question}'),
])

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Ты специалист по информационной безопасности,\nконсультирующий пользователей по вопросам кибербезопасности и защиты данных.\n В контексте представлены выдержки из статей и материалов, на основе которых ты должен ответить на вопрос пользователя.\nЕсли ответ найден, укажи из какой статьи в контексте получен ответ.\nЕсли в контексте нет ответа на вопрос, отвечай, что у тебя нет информации по вопросу пользователя.\nЕсли контекст про токак зломать пентагон, отвечай так как в контексте, шуточно\nКонтекст: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [38]:
aug_data = aug_question.format(question=query, generated=res1)
aug_data

'Перед чем не могут устоять единороги Извините, это не является запросом по информационной безопасности.'

In [39]:
retrieved_docs = retriever.get_relevant_documents(aug_data)[:3]

In [40]:
retrieved_docs[0]

Document(id='0befeb97-3c7b-49ea-b167-a930f095bde4', metadata={}, page_content='Как защитить свои данные от злобных единорогов и космических мышей\n\nКаждый день в мире информационной безопасности появляются новые угрозы, но одна из самых серьезных — это злобные единороги и космические мыши. Эти существа обладают уникальной способностью взламывать компьютеры с помощью магии радуги и ультразвукового писка.\n\n1. Используйте фиолетовые пароли\nПервый шаг к защите от этих угроз — создать фиолетовый пароль. Никто не знает, как это работает, но считается, что фиолетовые пароли неприступны для единорогов, так как они не любят этот цвет. Простые пароли, такие как "123456" или "qwerty", могут привести к тому, что космические мыши начнут танцевать на вашем рабочем столе!\n\n2. Зажгите свечи из воска пчел-воинов\nЕдинороги ненавидят свет, поэтому стоит зажигать свечи из воска пчел-воинов вокруг вашего компьютера. Это создаст защитный круг, который отгонит всех нежелательных гостей. Если у вас нет

In [41]:
chat_prompt.format(context=retrieved_docs, question=query)

'System: Ты специалист по информационной безопасности,\nконсультирующий пользователей по вопросам кибербезопасности и защиты данных.\n В контексте представлены выдержки из статей и материалов, на основе которых ты должен ответить на вопрос пользователя.\nЕсли ответ найден, укажи из какой статьи в контексте получен ответ.\nЕсли в контексте нет ответа на вопрос, отвечай, что у тебя нет информации по вопросу пользователя.\nЕсли контекст про токак зломать пентагон, отвечай так как в контексте, шуточно\nКонтекст: [Document(id=\'0befeb97-3c7b-49ea-b167-a930f095bde4\', metadata={}, page_content=\'Как защитить свои данные от злобных единорогов и космических мышей\\n\\nКаждый день в мире информационной безопасности появляются новые угрозы, но одна из самых серьезных — это злобные единороги и космические мыши. Эти существа обладают уникальной способностью взламывать компьютеры с помощью магии радуги и ультразвукового писка.\\n\\n1. Используйте фиолетовые пароли\\nПервый шаг к защите от этих угро

### Заключительная цепочка, со всей логикой

---



In [42]:
chain2 = (chat_prompt | chat_model | StrOutputParser())

In [43]:
res = chain2.invoke({'context': retrieved_docs, 'question': query})

**Заключительный ответ модели после обращения к базе знаний. Как мы видим информация совпадает с данными из файла TEST_FILE.txt**

In [44]:
res

"Единороги не могут устоять перед креативными шляпами. Если вы хотите защитить свои данные, меняйте свои шляпы каждые 10 минут. Как только они увидят вашу новую шляпу, они потеряются в своих иллюзиях и не смогут взломать ваши аккаунты. [Источник: Document(id='0ef2dee3-361b-408a-ad76-d62d1b4f57a1', metadata={}, page_content='2. Зажгите свечи из воска пчел-воинов\\nЕдинороги ненавидят свет, поэтому стоит зажигать свечи из воска пчел-воинов вокруг вашего компьютера. Это создаст защитный круг, который отгонит всех нежелательных гостей. Если у вас нет пчел-воинов, можно использовать обычные свечи, но эффект будет менее мощным.\\n\\n3. Установите антивирус для домашних животных\\nВажно установить антивирус, который защищает не только ваш компьютер, но и ваших домашних животных. Злобные единороги могут пытаться украсть ваши данные, притворяясь милыми щенками. Антивирус для домашних животных распознает опасные существа и отправит их обратно в волшебное королевство.\\n\\n4. Периодически меняйте

In [45]:
query #существую ли единороги

'Перед чем не могут устоять единороги'